<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 7: Correlations</h1>


<a name='section_7_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_1">L7.1 Understanding Best Fit (Revisited)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_1">L7.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_2">L7.2 Minimizing on a Surface (1D Scan)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_2">L7.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_3">L7.3 Minimizing on a Surface (2D Scan)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_3">L7.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_4">L7.4 Correlations Between Fit Parameters: Part 1</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_4">L7.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_5">L7.5 Correlations Between Fit Parameters: Part 2</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_5">L7.5 Exercises</a></td>
    </tr>
</table>



<h3>Learning Objectives</h3>

In this lecture, we wil explore the nature of correlated variables and how they impact our interpretation of results. Furthermore, we will understand how to go away from correlations to see how we can reduce their impact. Additionally, we will start to scan the parameters of our fit beyond just finding the minimum. By scanning, we start to learn the subtleties of each of these setups, and we start to get deep insight into the nature of what we are minimizing and how to interpret it. 


<h3>Importing Data (Colab Only)</h3>

If you are in a Google Colab environment, run the cell below to import the data for this notebook. Otherwise, if you have downloaded the course repository, you do not have to run the cell below.

See the source and attribution information below:

>data: data/L05/events_a4_1space.dat, data/L05/events_a8_1space.dat <br>
>source: https://www.auger.org/index.php/science/data <br>
>source : https://arxiv.org/abs/1709.07321 <br>
>attribution: Pierre Auger Collaboration, arXiv:1709.07321v1 <br>
>license type: https://creativecommons.org/licenses/by-sa/4.0/ 

In [ ]:
#>>>RUN: L7.0-runcell00

#importing data from git repository

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L05' >> .git/info/sparse-checkout
!git pull origin main

<h3>Importing Libraries</h3>

Before beginning, run the cells below to import the relevant libraries for this notebook.

In [ ]:
#>>>RUN: L7.0-runcell01

# for Colab users
!pip install lmfit

In [ ]:
#>>>RUN: L7.0-runcell02

import numpy as np                 #https://numpy.org/doc/stable/
from scipy import optimize as opt  #https://docs.scipy.org/doc/scipy/reference/optimize.html
import matplotlib.pyplot as plt    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import math                        #https://docs.python.org/3/library/math.html
import csv                         #https://docs.python.org/3/library/csv.html
import lmfit                       #https://lmfit.github.io/lmfit-py/ 
import scipy.stats as stats        #https://docs.scipy.org/doc/scipy/reference/stats.html
from scipy.optimize.zeros import RootResults        #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.RootResults.html
from scipy.optimize.optimize import OptimizeResult  # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html
from typing import Callable, Tuple #https://docs.python.org/3/library/typing.html#typing.Callable
                                   #https://docs.python.org/3/library/typing.html
import numpy.linalg as la          #https://docs.scipy.org/doc/scipy/reference/linalg.html
from matplotlib.patches import Ellipse  # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.patches.Ellipse.html


<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: L7.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_7_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.1 Understanding Best Fit (Revisited)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_0) | [Exercises](#exercises_7_1) | [Next Section](#section_7_2) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2022/block-v1:MITxT+8.S50.1x+3T2022+type@sequential+block@seq_LS7/block-v1:MITxT+8.S50.1x+3T2022+type@vertical+block@vert_LS7_vid1" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_01.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L7.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_01.html', width=970, height=550)

<h3> What is the Chi-squared-value from a p-value? </h3>

So what is the importance of this fit? This is just equal to a $\chi^{2}$ distribution, as we have seen before. Before we delve into the details of the importance of the fit, let's just remind ourselves how to compute the p-value from a gaussian and the $\chi^{2}$ value from the p-value or a sigma value assuming a Gaussian. 


In [ ]:
#>>>RUN: L7.1-runcell01

import scipy.stats as stats

sigma = 1
print(stats.norm.cdf(sigma)-stats.norm.cdf(-sigma))

def pval(iVal):
    return stats.norm.cdf(iVal)-stats.norm.cdf(-iVal)

def chi2Val(iGausSigma,iNDOF):
    val=stats.chi2.ppf(pval(iGausSigma),iNDOF)
    return val

print(chi2Val(1,2))
#print(chi2Val(1,1))
#print(chi2Val(2,1))
#print(chi2Val(3,1))
#print(chi2Val(4,1))

<a name='exercises_7_1'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_1) | [Next Section](#section_7_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.1.1</span>

Compute the Chi-square value for 2$\sigma$ and 2 degrees of freedom. Enter your answer as a number with precision 1e-2. 


In [ ]:
#>>>EXERCISE: L7.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.


<a name='section_7_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.2 Minimizing on a Surface (1D Scan)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_1) | [Exercises](#exercises_7_2) | [Next Section](#section_7_3) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2022/block-v1:MITxT+8.S50.1x+3T2022+type@sequential+block@seq_LS7/block-v1:MITxT+8.S50.1x+3T2022+type@vertical+block@vert_LS7_vid2" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_02.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L7.2-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_02.html', width=970, height=550)

<h3>Uncertainty on more than one parameter</h3>

Previously, we explored fitting more than one parameter. That is, we minimized some error metric for more than one parameter. Let's take a closer look. As before, we'll fit the function 

$$
\begin{equation}
 f(x,a,b) = a + b \sin(x)
\end{equation}
$$

to the Auger data, i.e. finding optimal choices of parameters $a$ and $b$ given some set of sample observations $y_i$ and their sample uncertainties $\sigma_i$. One way to choose optimal parameter values is by optimizing our likelihood over both $a$ and $b$. As a reminder, likelihood defines  how probable our observed data is as a function of parameter values and a hypothesized form. At extrema, we'd expect our likelihood is minimized or 

$$
\begin{equation}
 \frac{\partial \mathcal{L}}{\partial a} = 0 \\
 \frac{\partial \mathcal{L}}{\partial b} = 0 \\
\end{equation}
$$

As we saw in L5.7, maxima of the likelihood function correspond to minima of the relevant $\chi^2$ statistic of the form

$$
\begin{equation}
  \chi^{2}(x|a,b) = \sum_{i=1}^{N} \frac{(y_{i}-f(x_{i},a,b))^2}{\sigma_i^2}
\end{equation}
$$

where we will make the assumption that $\sigma_i = \sqrt{f(x_{i},a,b)}$. At the extrema,

$$
\begin{eqnarray}
\frac{\partial \chi^{2}(x|a,b) }{\partial a} = 0 \\
\frac{\partial \chi^{2}(x|a,b) }{\partial b} = 0
\end{eqnarray}
$$

In a little bit, we're going to use this to manually write code to fit the function using the generic optimizer in `scipy`. But first, let's review how things look using `lmfit`. 

In [ ]:
#>>>RUN: L7.2-runcell01

############ This is all code from a previous lesson
import numpy as np
import csv
import math
import lmfit
import matplotlib.pyplot as plt
from scipy import optimize as opt

def rad(iTheta):
    return iTheta/180. * math.pi

def rad1(iTheta):
    return iTheta/180. * math.pi-math.pi

def load(label):
    dec=np.array([])
    ra=np.array([])
    az=np.array([])
    with open(label,'r') as csvfile:
        plots = csv.reader(csvfile,delimiter=' ')
        for pRow in plots:
            if '#' in pRow[0] or pRow[0]=='':
                continue
            dec = np.append(dec,rad(float(pRow[2])))
            ra  = np.append(ra,rad1(float(pRow[3])))
            az  = np.append(az,rad(float(pRow[4])))
    return dec,ra,az

def prephist(iRA):
    y0, bin_edges = np.histogram(iRA, bins=30)
    x0 = 0.5*(bin_edges[1:] + bin_edges[:-1])
    y0 = y0.astype('float')
    return x0,y0,1./(y0**0.5)

label8='data/L05/events_a8_1space.dat'

dec,ra8,az=load(label8)
xhist,yhist,xweights=prephist(ra8)


########## Tlast fit code

def fnew(x,a,b):
    pVal=b*np.sin(x)
    return a+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=10)
result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()


<h3>Re-run the Fit</h3>

Now, what we are going to do is re-run the fit. Here we'll go through the $\chi^2$ minimization process semi-manually, using `scipy.opt`. The code is explained in the Lesson video.

Strategically, what we are trying to do is write the $\chi^{2}$ value and use our normal minimizer tools to find the minimum parameters. 


In [ ]:
#>>>RUN: L7.2-runcell02

#This is the new code
def chi2(iX):
    #Note that this function depends on xhist and yhist being defined already.
    #This is bad practice in general, but we do it here for convenience.
    #After all, this code isn't reused elsewhere.
    #MAKE SURE YOU RUN prephist(ra8) to define yhist
    assert len(iX) == 2
    lTot=0
    for val in range(len(yhist)):
        xtest=fnew(xhist[val],iX[0],iX[1])
        lTot += (1./(xtest+1e-5))*(yhist[val]-xtest)**2
    return lTot

#First we minimize
x0 = np.array([1000,10]) # initial conditions
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

<h3>Visualizing the Uncertainty</h3>

Now, given that we have found the minimum, let's look at how the $\chi^{2}$ value changes from the minimum. The important thing is to remember that from <a href="https://en.wikipedia.org/wiki/Wilks%27_theorem" target="_blank">Wilk's Thereom</a>, the uncertainty of a parameter can be obtained by taking $\Delta \chi^{2}$ from the minimum value to be equal to 1. Let's go ahead and plot the minimum and look at what $\Delta \chi^{2}=1$ looks like. 

Note, where the red and blue lines cross corresponds to the one $\sigma$ fluctuation up and down from the minimum.

In [ ]:
#>>>RUN: L7.2-runcell03

#Scan near the minimum of each value
x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.5,sol.x[1]*1.5, 100)

#Now let's fix one parameter at the minimum, and profile the other
plt.plot(x, chi2([x,sol.x[1]]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("a-value")
plt.ylabel("$\chi^{2}$")
plt.show()

#Now for the other parameter
plt.plot(y, chi2([sol.x[0],y]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("b-value")
plt.ylabel("$\chi^{2}$")
plt.show()

<h3>Computing the Best Fit and Uncertainty</h3>

We can code up this algorithm and compute the best fit and the uncertainty on the best fit parameters. Let's go ahead and code this up. 

In [ ]:
#>>>RUN: L7.2-runcell04

######## Now let's use a numerical solver to find the points at which a function crosses zero (root solver)
def chi2minX(xval, delta_chi2=1):
    val=chi2([xval,sol.x[1]])
    minval=chi2(sol.x) + delta_chi2
    return val-minval

def chi2minY(yval, delta_chi2=1):
    val=chi2([sol.x[0],yval])
    minval=chi2(sol.x) + delta_chi2
    return val-minval

def chi2uncX(sol):
    solX1=opt.root_scalar(chi2minX,bracket=[sol.x[0], sol.x[0]*1.02],method='brentq')
    solX2=opt.root_scalar(chi2minX,bracket=[sol.x[0]*0.98, sol.x[0]],method='brentq')
    print("a:",sol.x[0],"+/-",abs(solX2.root-solX1.root)/2.)
    print("Reminder the Poisson uncertainty would be:",math.sqrt(sol.x[0]/40))
    return solX1, solX2

def chi2uncY(sol):
    solY1=opt.root_scalar(chi2minY,bracket=[sol.x[1],    sol.x[1]*1.2],method='brentq')
    solY2=opt.root_scalar(chi2minY,bracket=[sol.x[1]*0.8, sol.x[1]],method='brentq')
    print("b:",sol.x[1],"+/-",abs(solY2.root-solY1.root)/2.)
    return solY1, solY2

solX1, solX2 = chi2uncX(sol)
solY1, solY2 = chi2uncY(sol)

<h3>So what have we done?</h3>

We've used the optimizer to minimize the $\chi^{2}$ values in 2D, and we have obtained an uncertainty by looking at $\Delta \chi^{2}$. You can see that our semi-manual manipulation got us to the same parameter estimates as the `lmfit` example. Additionally, by looking at slices of $\chi^{2}$ in each of the parameters, we obtained the same uncertainties. 

As a side note, our final fitted uncertainty is a little bit larger than the Poisson uncertainty. In fact, for all fits there is a rule that our uncertainties on any parameter have to be larger than a certain number. This bound is known as the Cramér-Rao bound. I won't derive it or go into in detail, but the Cramér-Rao bound states.

$$
\begin{equation}
\mathrm{Var}(\theta|\hat{\theta}) \geq \frac{1}{\mathcal{I}(\theta)} \\
\mathcal{I}(\theta) = E_{p(X|\theta)}\left[-\frac{\partial^{2}}{\partial\theta^{2}}\log\left(p\left(x|\theta\right)\right)\right]
\end{equation}
$$

Where $\mathrm{Var}(\theta|\hat{\theta})$ is the variance on our fitted parameter and we call $\mathcal{I}(\theta)$ the <a href="https://en.wikipedia.org/wiki/Fisher_information" target="_blank">Fisher information</a>, which generalizes the variance over binomial sampled distributions and, in its simplest form, is equal the inverse of the variance of a binomial distribution, which in turn is the more precise description of a poisson distribution. Hence the fact that our uncertainty is larger than the Poisson uncertainty. While I don't want to go into this more, this result is powerful because it means that there is limit to when you should stop searching for a best fit. 

<a name='exercises_7_2'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_2) | [Next Section](#section_7_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.2.1</span>

What are the 2$\sigma$ bounds (that is, 95.45% confidence interval values) of $a$ and $b$ in the fit above? Enter your answer as a list of numbers with precision 1 (the nearest whole number): `[a_lower, a_upper, b_lower, b_upper]`

In [ ]:
#>>>EXERCISE: L7.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#For a given parameter, assume everything else is determined, so we're
# working with 1 degree of freedom.
# First, we want to know the chi-square value at which 2 standard deivations (95.45% of the probability)
# in the chi-square distribution is for values less extreme than that value.
ndof=1
pval=0.9545
val = stats.chi2.ppf(pval,ndof)
print("Delta Chi-square:", val)

def minXfunc(x):
    return chi2minX(x, delta_chi2=val)

def minYfunc(x):
    return chi2minY(x, delta_chi2=val)

def chi2unc(sol, sol_index, min_func, unc_prop_guess):
     #insert code here
    return valmin,valmax

a1, a2 = chi2unc(sol, 0, minXfunc, 0.08)
b1, b2 = chi2unc(sol, 1, minYfunc, 0.4)

print(f"a bounds: [{a1}, {a2}]")
print(f"b bounds: [{b1}, {b2}]")

<a name='section_7_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.3 Minimizing on a Surface (2D Scan)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_2) | [Exercises](#exercises_7_3) | [Next Section](#section_7_4) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2022/block-v1:MITxT+8.S50.1x+3T2022+type@sequential+block@seq_LS7/block-v1:MITxT+8.S50.1x+3T2022+type@vertical+block@vert_LS7_vid3" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_03.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L7.3-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_03.html', width=970, height=550)

<h3>Overview</h3>
    
Now given our minimization worked in 1D for both variables $a$ and $b$, what about if we scan both variables simultaneously and then look at how the minimum behaves. To do this, we can start by first scaning around the best fit values of $a$ and $b$ and just plotting the value of our $\chi^{2}$ minimization. From this space, we can start to relate this back to the 2D version of Wilk's theorem in the slides above. 

To really visualize the whole thing let's make one more plot: how the $\chi^2$ value depends on both parameters, i.e., in 2D. We can do this using the `meshgrid` function and plotting it. In addition, we will add some color full lines for a $\Delta \chi^{2}$ given by various values, we will revisit what these values are in a little bit.

In [ ]:
#>>>RUN: L7.3-runcell01

#define the 2D X and Y grid
x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100) #grid in x
y = np.linspace(sol.x[1]*0.5,sol.x[1]*1.5, 100)#grid in y
X, Y = np.meshgrid(x, y) #2d grid

# For z coordinate, evaluate chi2 at each x,y point in the grid.
# note understanding this one-liner itself isn't too important
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)

#and plot
def plotColorsAndContours(X,Y,Z):
    fig, ax = plt.subplots(1, 1)
    c = ax.pcolor(X,Y,Z,cmap='RdBu')
    cb=fig.colorbar(c, ax=ax)
    plt.xlabel("a")
    plt.ylabel("b")
    cb.set_label("$\chi^{2}$")
    #Now let's plot the contours of Delta chi^2
    levels = [0.1,1,2.3,4,6.18,9, 16, 25, 36, 49, 64, 81, 100]
    for i0 in range(len(levels)):
        levels[i0] = levels[i0]+sol.fun
    c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
    #plt.show()
    
plotColorsAndContours(X,Y,Z)

In the plot above we've shown curves representing different $\chi^2$ increases from the minimum. In the last section, we obtained uncertainties for our fit parameters by allowing just one of them to vary, and then observing the $\Delta\chi^2$. We may ask, what is the uncertainty profile when we are letting two parameters float simultaneously?

Let's go back to the Taylor expansion result in the Lesson slides, which led to the relation of Wilk's theorem, but write the expansion in terms of all variables $\vec{\theta}$:  



$$
\begin{equation}
\chi^{2}(x_{i},\vec{\theta})=\chi^{2}_{min}(x_{i},\vec{\theta})+\frac{1}{2}(\theta_{i}-\theta_{j})^{T}\frac{\partial^{2}}{\partial \theta_{i}\theta_{0}}\chi^{2}_{min}(x_{i},\vec{\theta}_{0})(\theta_{j}-\theta_{0})
\end{equation}
$$

We can write this out in 2D as:

$$
\begin{equation}
\chi^{2}(x,\vec{\theta})=\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\theta_{a}-\theta_{a-min} & \theta_{b}-\theta_{b-min}\end{array}\right)\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\\
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\left(\begin{array}{c}
\theta_{a}-\theta_{a-min}\\
\theta_{b}-\theta_{b-min}
\end{array}\right)
\end{equation}
$$

In the case where $\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\approx0$ we can simplify this approximation a lot:

$$
\begin{equation}
\chi^{2}(x,\vec{\theta})=\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\Delta\theta_{a} & \Delta\theta_{b}\end{array}\right)\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & 0\\
0 & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\left(\begin{array}{c}
\Delta\theta_{a}\\
\Delta\theta_{b}
\end{array}\right)
\end{equation}
$$

This all becomes a 2D quadratic equation:

$$
\begin{align}
\chi^{2}(x,\vec{\theta}) & =\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\Delta\theta_{a}^{2}\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}}+ & \Delta\theta_{b}^{2}\frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}\end{array}\right)\\
 & =\chi_{min}^{2}(x,\vec{\theta})+\left(\begin{array}{cc}
\frac{\Delta\theta_{a}^{2}}{\sigma_{\theta_{a}}^{2}}+ & \frac{\Delta\theta_{b}^{2}}{\sigma_{\theta_{b}}^{2}}\end{array}\right)
\end{align}
$$

I want to point out, we are now profiling two parameters at once in this 2D plot, which means the contribution to $\chi^{2}$ involves 2 degrees of freedom. You can see this from the above equation, since its the sum of 2 gaussian distributed variables with width $1$ and mean $0$. The 1 $\sigma$ confidence interval for 2 degrees of freedom is computed by taking $\Delta \chi^2(x,\nu=2)=x~\mathrm{where~}\mathrm{cdf}\left(\chi^{2}(x,2)=0.683\right)\approx2.3$. This is the yellow contour. Contrast this with the case of 1 $\sigma$ confidence on 1 degree of freedom, where $\Delta\chi^2 = 1$. 

Now, we can go one step further, and note that the above formula  $ a x^2 + b y^2 = c$ is just the form of an ellipse with a specific major and minor axis. As a result, we can define a 3D funciton given by 

$$ f(x,y) = \left(\frac{x-\bar{a}}{\sigma_{a}}\right)^{2} + \left(\frac{y-\bar{b}}{\sigma_{b}}\right)^{2} $$

Anyway, let's plot the ellipses and compare it to our minimum contours. 


In [ ]:
#>>>RUN: L7.3-runcell02

#Let's plot the uncertainties  from hess_inv
print(np.sqrt(2*sol.hess_inv))
#the diagonals are approximately the errors

#Make a the expression in the above equation x and x0 are 2 vectors
def quadratic2D(x,x0,sigma0):
    lVals=x-x0
    lVals=(lVals**2)/(sigma0)/sigma0
    return np.sum(lVals)

plotColorsAndContours(X,Y,Z)

#Now plot the ellipse in 3D
def plotEllipse(sigx,sigy):
    levels = [0.1,1,2.3,4,6.18,9, 16, 25, 36, 49, 64, 81, 100]
    ZQ = np.array([quadratic2D([x,y],sol.x,[sigx,sigy]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
    c = plt.contour(X, Y, ZQ, levels,colors=['red', 'blue', 'yellow','green'],linestyles='dashed')

sigx=(solX2.root-solX1.root)/2.
sigy=(solY2.root-solY1.root)/2.
plotEllipse(sigx,sigy)
plt.show()

Remarkably, our space, give by the formula above, yields the solid lines, which matches very closely to the dashed contours obtained by just writing the $\chi^{2}$ value. This is a profound statement, it really is. 

This gives us the same yellow line for our 2-variable 1$\sigma$ confidence ellipse. If you look very closely, you will see a small difference. This observation raises the question of what happens when $\frac{\partial^{2}\chi^{2}}{\partial \theta_{a}\theta_{b}}\neq0$. We'll see an example of that when we look at correlations in fitting a different set of parameters.


<a name='exercises_7_3'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_3) | [Next Section](#section_7_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.3.1</span>

When we allow 2 parameters to vary, for a given confidence level (e.g. 1$\sigma$) we end up with a confidence ellipse containing parameter values outside the confidence intervals on the individual parameters alone. Compute the 1$\sigma$ (68.27%) confidence interval bounds for the parameter $a$, based on this ellipse from floating both $a$ and $b$.

Enter your answer as a list of number with precision 1 (nearest whole number): `[a_lower, a_upper]`

Why do we typically quote the uncertainty from the 1D variation?

In [ ]:
#>>>EXERCISE: L7.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

# We can follow the same procedure for finding the confidence interval based on 1D
# but for 1-sigma we now use 2.3 
# Use the code from the solution to Ex. 7.2.1,
ndof=#insert value here
pval=0.6827#1 sigma p-value
val = stats.chi2.ppf(pval,ndof) # 2 DoF this time!
print("Delta Chi-square:", val)

def minfunc(x):
    return chi2minX(x, delta_chi2=val)

a1, a2 = #YOUR CODE HERE

print(f"a bounds: [{a1}, {a2}]")

<a name='section_7_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.4 Correlations Between Fit Parameters: Part 1</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_3) | [Exercises](#exercises_7_4) | [Next Section](#section_7_5) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2022/block-v1:MITxT+8.S50.1x+3T2022+type@sequential+block@seq_LS7/block-v1:MITxT+8.S50.1x+3T2022+type@vertical+block@vert_LS7_vid4" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

 <h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_04.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L7.4-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_04.html', width=970, height=550)

<h3>Overview</h3>
    
Ok, so now let's try to understand what happens when our off-diagonal terms are off. Let's go back to Wilk's theorem and look at the Hessian for the $\chi^{2}$, but now with non-zero off-diagonal parameters.

To understand the point of this, what we are going to do is make the 2D contour plot like the one above with the same contours as before, but now we will use a new function, that is just a reparametrization of the old one. This function will be

$$
\begin{equation}
 f(x) = a x + b (1-x)
\end{equation}
$$

This means that the values of $a$ and $b$ now have different meaning, but this is just a linear fit like before and it should give the same overall $\chi^{2}$ value as before. 


In [ ]:
#>>>RUN: L7.4-runcell01

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=1000)

result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()

x0 = np.array([1000,1000])
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
plotColorsAndContours(X,Y,Z)
plt.show()

Now critically, let's now make the same ellipse plot with our new best fit values for $a$ and $b$, and our new uncertainties. Additionally, we can also do the 1D uncertainty scans like we did above again, to see what uncertainties we get on $a$ and $b$. 

In [ ]:
#>>>RUN: L7.4-runcell02

plotColorsAndContours(X,Y,Z)
solX1, solX2 = chi2uncX(sol)
solY1, solY2 = chi2uncY(sol)
sigx=(solX2.root-solX1.root)/2.
sigy=(solY2.root-solY1.root)/2.
plotEllipse(sigx,sigy)
plt.show()


What we can see is that our ellipses differ pretty dramatically from above. Also, we see that our 1D uncertainties are very mis-estimated from what lmfit gives us. The reason for this is that `lmfit` takes into account the full correlation of the variables, whereas just doing a 1D scan is equivalent to just moving on a single line along the $y$ and $x$ axes. 

<a name='exercises_7_4'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_4) | [Next Section](#section_7_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.4.1</span>

Run the previous fit with just a regular linear fit given by $f(x) = ax + b$. How does the $\chi^{2}$ change? How do the correlations between the variables change? Choose the best option from the following:

- $\chi^{2}$ increases and the correlations increase
- $\chi^{2}$ decreases and the correlations increase
- $\chi^{2}$ stays approximately the same and the correlations increase
- $\chi^{2}$ increases and the correlations decrease
- $\chi^{2}$ decreases and the correlations decrease
- $\chi^{2}$ stays approximately the same and the correlations decrease
- $\chi^{2}$ increases and the correlations stay approximately the same
- $\chi^{2}$ decreases and the correlations stay approximately the same
- $\chi^{2}$ stays approximately the same and the correlations stay approximately the same

In [ ]:
#>>>EXERCISE: L7.4.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.



<a name='section_7_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.5 Correlations Between Fit Parameters: Part 2</h2>     

| [Top](#section_7_0) | [Previous Section](#section_7_4) | [Exercises](#exercises_7_5) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2022/block-v1:MITxT+8.S50.1x+3T2022+type@sequential+block@seq_LS7/block-v1:MITxT+8.S50.1x+3T2022+type@vertical+block@vert_LS7_vid5" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

Now if we go back to our original correlated parameterization, we see that the 1D profile method for obtaining the uncertianties is not the same as the uncertainties we get from the fit.  This is clearly a problem, how can we address this issue? 

<h3>Dealing with correlated uncertainties</h3>

Looking at our parameters, our uncertainty estimate is smaller than what we observed above. The uncertainties quoted now differ from what we got by varying from the $\chi^{2}$ minimum. What we are doing is moving up and down the 2D plot and looking at $\Delta \chi^{2}$. However, the two parameters are strongly correlated, meaning that the value of one parameter that minimizes the $\chi^{2}$ changes as the value of the other parameter changes. Because of that, you see that this doesn't capture the true uncertainty in the sense that we can move further along $x$ or $y$ (while changing the value of the other parameter appropriately) and still be within the yellow or even blue ellipse. This is quite clear when you overlay the uncertainty from the quadratic function, which draws circles not ellipses. 

What, then, is the right uncertainty to quote? 

Notice our fit with `lmfit` outputs a parameter $C(a,b)=0.872$. This is, in fact, a measure of the correlation between the two parameters $a$ and $b$. We also see from our optimization function that we get something labelled as the `hess_inv`. This we can write noting the relation of the $\chi^{2}$ Hessian and uncertainties as:

$$
\begin{equation}
\chi^{2}(x_{i},\vec{\theta})=\chi^{2}_{min}(x_{i},\vec{\theta})+\frac{1}{2}(\theta_{i}-\theta_{j})^{T}\frac{\partial^{2}}{\partial \theta_{i}\theta_{0}}\chi^{2}_{min}(x_{i},\vec{\theta}_{0})(\theta_{j}-\theta_{0})
\end{equation}
$$

which allows us to write the uncertainties as:

$$
\begin{equation}
 \frac{2}{\sigma^{2}}=\frac{\partial^{2}\chi^{2}}{\partial\theta_{i}\partial\theta_{j}} \\
 \sigma^{2}    = 2\left(\frac{\partial^{2}\chi^{2}}{\partial\theta_{i}\partial\theta_{j}}\right)^{-1} 
\end{equation}
$$

Now, in the case where the off-diagonals of the Hessian were zero, the parameters were uncorrelated, so:

$$
\begin{equation}
\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & 0\\
0 & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\rightarrow\left(\begin{array}{cc}
\frac{2}{\sigma_{a}^{2}} & 0\\
0 & \frac{2}{\sigma_{b}^{2}}
\end{array}\right)
\end{equation}
$$

But here we have something more complicated. However, this is a natural way to define correlations. Let's first verify our intuition for our minimization scheme by taking our $2\times2$ Hessian matrix and diagonalizing, computing the eigenvectors and the eigenvalues. We can diagonalize the matrix as: 


$$
\begin{equation}
A^{-1}=2\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\\
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)^{-1}A=\left(\begin{array}{cc}
\sigma_{1}^{2} & 0\\
0 & \sigma_{2}^{2}
\end{array}\right)
\end{equation}
$$

It is important to note that for any N dimensional Hessian, provided the determinant is not zero, we can find a basis of independent variables that are not correlated. That is to say we can always diagonalize our Hessian, and the eigenvectors of our Hessian are the independent values with variances given by the eigenvalues. These are our true uncertainty values (eigenvalues) and direction of variation (eigenvector).  

When we run our minimizer, we get the Hessian inverse, which we can play with. Let's go ahead and look at the Hessian from our minimizer. From the Hessian, we can import numpy's linear algebra toolkit and compute the eigen vectors and values. Finally, we can draw these vectors and values on our 2D scan plot above. Let's do it:


<h3>Rerun correlated fit</h3>

First, let's rerun the correlated fit.

In [ ]:
#>>>RUN: L7.5-runcell01

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=1000)

result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()

x0 = np.array([1000,1000])
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
plotColorsAndContours(X,Y,Z)
plt.show()


In [ ]:
#>>>RUN: L7.5-runcell02

print(np.sqrt(2*sol.hess_inv))
#The diagonals are the uncertainty lmfit quotes

#Really the best way to do this is to get the eigen values using an linear algebra problem
import numpy.linalg as la
w, v=la.eig(2*sol.hess_inv)
print("values",w,"vectors",v)

#Now let's plot the eigenvectors
from matplotlib.patches import Ellipse
def get_cov_ellipse(cov, centre, nstd, **kwargs):
    """
    Return a matplotlib Ellipse patch representing the covariance matrix
    cov centred at centre and scaled by the factor nstd.

    """
    # Find and sort eigenvalues and eigenvectors into descending order
    eigvals, eigvecs = np.linalg.eigh(cov)
    order = eigvals.argsort()[::-1]
    eigvals, eigvecs = eigvals[order], eigvecs[:, order]

    # The anti-clockwise angle to rotate our ellipse by 
    vx, vy = eigvecs[:,0][0], eigvecs[:,0][1]
    theta = np.arctan2(vy, vx)

    # Width and height of ellipse to draw
    width, height = 2 * nstd * np.sqrt(eigvals)
    return Ellipse(xy=centre, width=width, height=height,angle=np.degrees(theta), **kwargs)

err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x,1)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
for i0 in range(len(levels)):
    levels[i0] = levels[i0]+sol.fun
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.show()

Now the filled in circles make a direct correspondance with the uncertainties above. 

From the above, we can now formulate a description of uncertainties in our system. When we had uncorrelated parameters, we had a total uncertainty from our $\chi^{2}$ given by 


$$
\begin{equation}
\sigma_{tot}^{2} = \sigma_{a}^2+\sigma_{b}^2
\end{equation}
$$

Now, we have the full ellipse

$$
\begin{equation}
\sigma_{tot}^{2} = \sigma_{a}^2+\sigma_{b}^2+2\sigma_{ab}
\end{equation}
$$

where $\sigma_{ab}=\rm{COV(a,b)}$. There are a number of names for this variable, but they are all equivalent. Let's write it out. We can write the error matrix in 2D as:


$$
\begin{equation}
\left(\begin{array}{cc}
\sigma_{a}^{2} & {\rm COV}(a,b)\\
{\rm COV}(a,b) & \sigma_{b}^{2}
\end{array}\right)=\sum_{i=1}^{N}\left(\begin{array}{cc}
\left(a_{i}-\bar{a}\right)^{2} & \left(a_{i}-\bar{a}\right)\left(b_{i}-\bar{b}\right)\\
\left(a_{i}-\bar{a}\right)\left(b_{i}-\bar{b}\right) & \left(b_{i}-\bar{b}\right)^{2}
\end{array}\right)
\end{equation}
$$

where on the right side we have written it in terms of the computation over events. Recall that, for a linear regression, the slope is just the $\rm{COV(x,y)/VAR(x)}$, so the covariance matrix is intricately tied with the slope.  

We can also write it as the correlation matrix where we normalize by the uncertainties:

$$
\begin{equation}
\rho=\left(\frac{1}{\sigma_{a}}  \frac{1}{\sigma_{b}} \right)^{T}\left(\begin{array}{cc}
1 & \frac{{\rm COV}(a,b)}{\sigma_{a}\sigma_{b}}\\
\frac{{\rm COV}(a,b)}{\sigma_{a}\sigma_{b}} & 1
\end{array}\right)\left(\frac{1}{\sigma_{a}}   \frac{1}{\sigma_{b}} \right) 
\end{equation}
$$

Recall that the covariance is what we originally used to compute the linear slope of the points, this is exactly the same here. In fact, instead of scanning the likelihood analytically we could have sampled the points, and done a linear regression. The resulting slope and line can be related to our eigenvectors. One last thing to mention is that if variables are correlated, we can use the correlation to propagate the uncertainties. 

$$
\begin{equation}
\sigma_{f}^{2} = \left(\frac{\partial f}{\partial x}\right)^2\sigma_{x}^2 + \left(\frac{\partial f}{\partial y}\right)^{2}+\left(\frac{\partial f}{\partial x}\right)\left(\frac{\partial f}{\partial y}\right)\sigma_{xy}
\end{equation}
$$

Now let's check we can get the corelation matrix.

In [ ]:
#>>>RUN: L7.5-runcell03

#Now let's get the correlation C(a,b) (see below)
w, v=np.linalg.eig(2*sol.hess_inv)
print("c(a,b)",v[0,1]/v[0,0])
print("A deceptively wrong way to get correlation: since its not normalized",sol.hess_inv[0,1]/sol.hess_inv[0,0])

Finally, we are going to run our fit 

In [ ]:
#>>>RUN: L7.5-runcell04

import lmfit

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

#Randomly sample points in the above range
def maketoy(iy):
    toy=np.array([])
    #go through the y-values and Poisson fluctuate
    for i0 in range(len(iy)):
        pVal = np.random.normal (iy[i0],np.sqrt([iy[i0]]))
        toy = np.append(toy,float(pVal))
    return toy

def fittoy(ibin,iy):
    #generate toy
    toy=maketoy(iy)
    #now fit
    model  = lmfit.Model(fnew)
    p = model.make_params(a=1000,b=10)
    xweights=np.array([])
    #setup poison weight
    for i0 in range(len(toy)):
        xweights = np.append(xweights,1./math.sqrt(toy[i0]))
    result = model.fit(data=toy,x=ibin, params=p, weights=xweights)
    return result.params["a"].value,result.params["b"].value

ntoys=1000
lAs=np.array([])
lBs=np.array([])
for i0 in range(ntoys):
    pA,pB=fittoy(xhist,yhist)
    lAs = np.append(lAs,pA)
    lBs = np.append(lBs,pB)

err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x, 1)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.plot(lAs,lBs,c='black',marker='.',linestyle = 'None')
plt.show()

You can see that if we randomly sample from our distributions, then we get the ellipse variation in $A$ and $B$. The toys have a lot of useful elements. We can use it to get the variation in our parameters.  What we can also do is look at the variance of our parameters and compare it to the Hessian that we can diagonalize to get the fit. 

In [ ]:
#>>>RUN: L7.5-runcell05

#Now let's run a. linear regression
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

print("A:",lAs.mean(),"+/-",lAs.std())
print("B:",lBs.mean(),"+/-",lBs.std())
print("Cov:",covariance(lAs,lBs),"A Variance:",variance(lAs),"B Variance:",variance(lBs))
print("Check with Hessian:",2*sol.hess_inv)
print("Cor:",covariance(lAs,lBs)/math.sqrt(variance(lAs)*variance(lBs)),"A Variance:",1.,"B Variance:",1.)

<a name='exercises_7_5'></a>   

| [Top](#section_7_0) | [Restart Section](#section_7_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.5.1</span>

Repeat the previous analysis using the uncorrelated fit, $f(x) = a x + b$. Specifically, plot the ellipse and compute the uncertainties in $a$ and $b$? Do they correspond with lmfit? 


In [ ]:
#>>>EXERCISE: L7.5.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

